# Multi-Tier Web Application

In this exercise we will be using OpenTofu (an open-source fork of Terraform) and Oracle Cloud Infrastructure's Free Tier to deploy WordPress with public and private subnets. The webserver will be in the public subnet and be accessible from the internet, while the database server will be in a private subnet with no internet connectivity.

[OpenTofu](https://opentofu.org)

[Oracle Cloud Free Tier](https://www.oracle.com/au/cloud/free/)

<img src="img/infrastructure.png" width="100%" alt="Cloud Infrastructure" />


## Authentication

When beginning a session you will need to authenticate:

```bash
oci session authenticate
```
A session lasts one hour. Don't forget to refresh your session to avoid having to authenticate again.


In [ ]:
!oci session refresh --profile harleycalvert

# Set Up OCI Terraform

[Set Up OCI Terraform](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-provider/01-summary.htm)

## Install OpenTofu

In [ ]:
!snap install --classic opentofu

In [ ]:
!tofu -v

## Create RSA Keys

In [ ]:
!mkdir $HOME/.oci

In [ ]:
!openssl genrsa -out $HOME/.oci/oci_rsa_key.pem 2048

In [ ]:
!chmod 600 $HOME/.oci/oci_rsa_key.pem

In [ ]:
!openssl rsa -pubout -in $HOME/.oci/oci_rsa_key.pem -out $HOME/.oci/oci_rsa_key.pem.pub

In [ ]:
!(cd $HOME/.oci/ && ls)

In [ ]:
!cat $HOME/.oci/oci_rsa_key.pem.pub

Add the public key to your user account.

In the OCI Console's top navigation bar, click the Profile menu, and then go to My profile.
- Click API Keys.
- Click Add API Key.
- Select Paste Public Keys.
- Paste value from previous step, including the lines with BEGIN PUBLIC KEY and END PUBLIC KEY.
- Click Add.
- Copy the Configuration file preview.
- Paste the Configuration file preview into ~/.oci/config

You have now set up the RSA keys to connect to your OCI account.

## Add API Key-Based Authentication

In [ ]:
!mkdir tf-provider

## Gather Required Information
Collect the following credential information from the OCI Console.

- Tenancy OCID
- User OCID
- Fingerprint
- Region 
  - ap-melbourne-1

Collect the following information from your environment.

- Private Key Path 
  - /home/harley/.oci/oci_rsa_key.pem
  
Put the information in provider.tf like so:

In [ ]:
%%writefile ./tf-provider/provider.tf


provider "oci" {  
  tenancy_ocid = "<tenancy-ocid>"
  user_ocid = "<user-ocid>" 
  private_key_path = "<rsa-private-key-path>"
  fingerprint = "<fingerprint>"
  region = "<region-identifier>"
}

In [ ]:
!code ./tf-provider/provider.tf

In [ ]:
%%writefile ./tf-provider/availability-domains.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/data-sources/identity_availability_domains

# Tenancy is the root or parent to all compartments.
# For this, use the value of <tenancy-ocid> for the compartment OCID.

data "oci_identity_availability_domains" "ads" {
  compartment_id = "<tenancy-ocid>"
}

In [ ]:
!code ./tf-provider/availability-domains.tf

In [ ]:
%%writefile ./tf-provider/outputs.tf


# Output the "list" of all availability domains.
output "all-availability-domains-in-your-tenancy" {
  value = data.oci_identity_availability_domains.ads.availability_domains
}

In [ ]:
!code ./tf-provider/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-provider && tofu init) 

In [ ]:
!(cd tf-provider && ls -al) 

In [ ]:
!(cd tf-provider && tree .) 

In [ ]:
!(cd tf-provider && tofu validate)

In [ ]:
!(cd tf-provider && tofu plan -input=false)

In [ ]:
!(cd tf-provider && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-provider && tofu output)

# Create a Compartment

[Create a Compartment](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-compartment/01-summary.htm)

## Create Scripts

In [ ]:
!mkdir ./tf-compartment

In [ ]:
!cp ./tf-provider/provider.tf ./tf-compartment/provider.tf 

In [ ]:
%%writefile ./tf-compartment/compartment.tf


resource "oci_identity_compartment" "tf-compartment" {
    # Required
    compartment_id = "<tenancy-ocid>"
    description = "<your-description>"
    name = "<your-compartment-name>"
}

Replace your-compartment-name with a name of your choice.

In [ ]:
!code ./tf-compartment/compartment.tf

In [ ]:
%%writefile ./tf-compartment/outputs.tf


# Outputs for compartment
output "compartment-name" {
  value = oci_identity_compartment.tf-compartment.name
}

output "compartment-OCID" {
  value = oci_identity_compartment.tf-compartment.id
}

In [ ]:
!code ./tf-compartment/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-compartment && ls -al) 

In [ ]:
!(cd tf-compartment && tree .) 

In [ ]:
!(cd tf-compartment && tofu init) 

In [ ]:
!(cd tf-compartment && tofu validate)

In [ ]:
!(cd tf-compartment && tofu plan -input=false)

In [ ]:
!(cd tf-compartment && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-compartment && tofu output)

# Create a Virtual Cloud Network

[Create a Virtual Cloud Network](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-vcn/01-summary.htm)

[VCN (basics)](https://isaac-exe.gitbook.io/various-tutorials/tutorials/untitled/vcn-basics)

## Create a Basic Network

In [ ]:
!mkdir ./tf-vcn

In [ ]:
!cp ./tf-provider/provider.tf ./tf-vcn/provider.tf 

In [ ]:
%%writefile ./tf-vcn/vcn.tf


# https://github.com/oracle-terraform-modules/terraform-oci-vcn

resource "oci_core_vcn" "vcn" {
  # We still allow module users to declare a cidr using `vcn_cidr` instead of the now recommended `vcn_cidrs`, but internally we map both to `cidr_blocks`
  # The module always use the new list of string structure and let the customer update his module definition block at his own pace.
  cidr_blocks    = ["10.0.0.0/16"]
  compartment_id = var.compartment_id
  display_name   = "vcn"
  dns_label      = "vcn"
  is_ipv6enabled = false

  freeform_tags = {
    terraformed = "Please do not edit manually"
    module      = "oracle-terraform-modules/vcn/oci"
  }
    
  defined_tags  = null

  lifecycle {
    ignore_changes = [defined_tags, dns_label, freeform_tags]
  }
}

## Customise the Network

### Create a Security List for the Private Subnet

In [ ]:
%%writefile ./tf-vcn/private-security-list.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_security_list

resource "oci_core_security_list" "private-security-list"{

# Required
  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id

# Optional
  display_name = "security-list-for-private-subnet"
    
  
  egress_security_rules {
    stateless = false
    destination = "0.0.0.0/0"
    destination_type = "CIDR_BLOCK"
    protocol = "all" 
  }
    
 
  ingress_security_rules { 
    stateless = false
    source = "10.0.0.0/16"
    source_type = "CIDR_BLOCK"
    # Get protocol numbers from https://www.iana.org/assignments/protocol-numbers/protocol-numbers.xhtml TCP is 6
    protocol = "6"
    tcp_options { 
      min = 22
      max = 22
    }
  }
    
 
  ingress_security_rules { 
    stateless = false
    source = "10.0.0.0/16"
    source_type = "CIDR_BLOCK"
    # Get protocol numbers from https://www.iana.org/assignments/protocol-numbers/protocol-numbers.xhtml TCP is 6
    protocol = "6"
    tcp_options { 
      min = 3306
      max = 3306
    }
  } 
    
}

### Create a Security List for the Public Subnet

In [ ]:
%%writefile ./tf-vcn/public-security-list.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_security_list

resource "oci_core_security_list" "public-security-list"{

# Required
  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id

# Optional
  display_name = "security-list-for-public-subnet"
    
  
  egress_security_rules {
    stateless = false
    destination = "0.0.0.0/0"
    destination_type = "CIDR_BLOCK"
    protocol = "all" 
  }

 
  ingress_security_rules {
    stateless   = false
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"

    # Allow SSH (TCP port 22)
    protocol = "6"  # TCP
    tcp_options {
      min = 22
      max = 22
    }
  }

    
  ingress_security_rules {
    stateless   = false
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"

    # Allow HTTP (TCP port 80)
    protocol = "6"  # TCP
    tcp_options {
      min = 80
      max = 80
    }
  }

    
  ingress_security_rules {
    stateless   = false
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"

    # Allow HTTPS (TCP port 443)
    protocol = "6"  # TCP
    tcp_options {
      min = 443
      max = 443
    }
 }  


}

### Create a Private Subnet

In [ ]:
%%writefile ./tf-vcn/private-subnet.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_subnet
# https://docs.oracle.com/en-us/iaas/tools/terraform-provider-oci/5.24/docs/r/core_subnet.html

resource "oci_core_subnet" "vcn-private-subnet"{

  # Required
  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id
  cidr_block = "10.0.1.0/24"
 
  # Optional
  #route_table_id = oci_core_vcn.vcn.nat_route_id
  security_list_ids = [oci_core_security_list.private-security-list.id]
  display_name = "private-subnet"
  dns_label = "private"
  prohibit_public_ip_on_vnic = true
}

### Create a Public Subnet

In [ ]:
%%writefile ./tf-vcn/public-subnet.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_subnet
# https://docs.oracle.com/en-us/iaas/tools/terraform-provider-oci/5.24/docs/r/core_subnet.html

resource "oci_core_subnet" "vcn-public-subnet"{

  # Required
  compartment_id = var.compartment_id
  vcn_id         = oci_core_vcn.vcn.id
  cidr_block     = "10.0.0.0/24"
 
  # Optional
  route_table_id      = oci_core_route_table.public-route-table.id
  security_list_ids   = [oci_core_security_list.public-security-list.id]
  display_name        = "public-subnet"
  dns_label           = "public"
}

### Create Internet Gateway

In [ ]:
%%writefile ./tf-vcn/internet-gateway.tf


resource "oci_core_internet_gateway" "internet-gateway" {
  compartment_id = var.compartment_id
  display_name = "internet-gateway"
  vcn_id = oci_core_vcn.vcn.id
}

### Create NAT Gateway

We could use a NAT Gateway to provide outbound internet access to the private subnet. This could be useful for installing software and security updates. However, a NAT Gateway is not available in Oracle's Free Tier. So you can skip this section.

In [ ]:
%%writefile ./tf-vcn/nat-gateway.tf


resource "oci_core_nat_gateway" "nat-gateway" {
  compartment_id = var.compartment_id
  display_name = "nat-gateway"
  vcn_id = oci_core_vcn.vcn.id
}

### Create Private Subnet Route Table

You may skip this section.

In [ ]:
%%writefile ./tf-vcn/private-route-table.tf


resource "oci_core_route_table" "private-route-table" {
  compartment_id = var.compartment_id
  vcn_id         = oci_core_vcn.vcn.id
  display_name   = "private-route-table"

  route_rules {
    destination_type = "CIDR_BLOCK"
    destination       = "0.0.0.0/0"
    network_entity_id = oci_core_nat_gateway.nat-gateway.id
  }

}


### Create Public Subnet Route Table

In [ ]:
%%writefile ./tf-vcn/public-route-table.tf


resource "oci_core_route_table" "public-route-table" {

  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id
  display_name = "public-route-table"

  route_rules {
    destination_type = "CIDR_BLOCK"
    destination = "0.0.0.0/0"
    network_entity_id = oci_core_internet_gateway.internet-gateway.id
  }
}


### Variables

In [ ]:
%%writefile ./tf-vcn/variables.tf


variable "compartment_id" {
  # This is the ID of the compartment you created earlier.
  # Check your compartment output  
  description = "Compartment ID"    
  type        = string
  default     = "<compartment-ocid>"
}

In [ ]:
!code ./tf-vcn/variables.tf

### Outputs

In [ ]:
%%writefile ./tf-vcn/outputs.tf


# Outputs for private security list

output "private-security-list-name" {
  value = oci_core_security_list.private-security-list.display_name
}
output "private-security-list-OCID" {
  value = oci_core_security_list.private-security-list.id
}
    

# Outputs for public security list

output "public-security-list-name" {
  value = oci_core_security_list.public-security-list.display_name
}
output "public-security-list-OCID" {
  value = oci_core_security_list.public-security-list.id
}


# Outputs for private subnet

output "private-subnet-name" {
  value = oci_core_subnet.vcn-private-subnet.display_name
}
output "private-subnet-OCID" {
  value = oci_core_subnet.vcn-private-subnet.id
}


# Outputs for public subnet

output "public-subnet-name" {
  value = oci_core_subnet.vcn-public-subnet.display_name
}
output "public-subnet-OCID" {
  value = oci_core_subnet.vcn-public-subnet.id
}

In [ ]:
!code ./tf-vcn/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-vcn && ls -al) 

In [ ]:
!(cd tf-vcn && tree .) 

In [ ]:
!(cd tf-vcn && tofu init) 

In [ ]:
!(cd tf-vcn && tofu validate)

In [ ]:
!(cd tf-vcn && tofu plan -input=false)

In [ ]:
!(cd tf-vcn && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-vcn && tofu output)

# Create a Database Compute Instance

[Create a Compute Instance](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-compute/01-summary.htm)

## Create SSH Encryption Keys
Execute the following command in the terminal:
```bash
ssh-keygen -t rsa -b 2048 -C "" -f ~/.ssh/database_vm_key
```
## Create Scripts

In [ ]:
!mkdir tf-database_vm

In [ ]:
!cp ./tf-provider/provider.tf ./tf-database_vm/database_vm.tf 

In [ ]:
!cp ./tf-vcn/variables.tf ./tf-database_vm/variables.tf 

In [ ]:
!cp ./tf-provider/availability-domains.tf ./tf-database_vm/availability-domains.tf

In [ ]:
%%writefile ./tf-database_vm/database_vm.tf


resource "oci_core_instance" "database_server_instance" {
  # Required
  availability_domain = data.oci_identity_availability_domains.ads.availability_domains[0].name
  compartment_id = var.compartment_id
  shape = "VM.Standard.E2.1.Micro"
  source_details {
    # https://docs.oracle.com/en-us/iaas/images/ 
    # Use Canonical-Ubuntu-22.04 for your region 
    source_id = "<source-ocid>"
    source_type = "image"
  }

  # Optional
  display_name = "ubuntu-database-instance"
  create_vnic_details {
    assign_public_ip = false
    # Private subnet
    # Find this in your VCN outputs.  
    subnet_id = "<subnet-ocid>"
  }
    
  metadata = {
    ssh_authorized_keys = file("<ssh-public-key-path>")
  }
    
  preserve_boot_volume = false
}

In [14]:
!code ./tf-database_vm/database_vm.tf 

In [ ]:
%%writefile ./tf-database_vm/outputs.tf


# The "name" of the availability domain to be used for the compute instance.
output "name-of-first-availability-domain" {
  value = data.oci_identity_availability_domains.ads.availability_domains[0].name
}

# Outputs for compute instance
output "public-ip-for-compute-instance" {
  value = oci_core_instance.database_server_instance.public_ip
}

output "private-ip-for-compute-instance" {
  value = oci_core_instance.database_server_instance.private_ip
}

output "instance-name" {
  value = oci_core_instance.database_server_instance.display_name
}

output "instance-OCID" {
  value = oci_core_instance.database_server_instance.id
}

output "instance-region" {
  value = oci_core_instance.database_server_instance.region
}

output "instance-shape" {
  value = oci_core_instance.database_server_instance.shape
}

output "instance-state" {
  value = oci_core_instance.database_server_instance.state
}

output "instance-OCPUs" {
  value = oci_core_instance.database_server_instance.shape_config[0].ocpus
}

output "instance-memory-in-GBs" {
  value = oci_core_instance.database_server_instance.shape_config[0].memory_in_gbs
}

output "time-created" {
  value = oci_core_instance.database_server_instance.time_created
}

In [ ]:
!code ./tf-database_vm/outputs.tf 

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-database_vm && ls -al) 

In [ ]:
!(cd tf-database_vm && tree .) 

In [ ]:
!(cd tf-database_vm && tofu init) 

In [ ]:
!(cd tf-database_vm && tofu validate)

In [ ]:
!(cd tf-database_vm && tofu plan -input=false)

In [ ]:
!(cd tf-database_vm && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-database_vm && tofu output)

## Connect to the Database Instance
To connect to the database, you must first connect to the webserver:
```bash
ssh -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
```

# Create a Webserver Compute Instance

[Create a Compute Instance](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-compute/01-summary.htm)
## Create SSH Encryption Keys
Execute the following command in the terminal:
```bash
ssh-keygen -t rsa -b 2048 -C "" -f ~/.ssh/webserver_vm_key
```
## Create Scripts

In [ ]:
!mkdir tf-webserver_vm

In [ ]:
!cp ./tf-provider/provider.tf ./tf-webserver_vm/provider.tf 

In [ ]:
!cp ./tf-vcn/variables.tf ./tf-webserver_vm/variables.tf 

In [ ]:
!cp ./tf-provider/availability-domains.tf ./tf-webserver_vm/availability-domains.tf

In [ ]:
%%writefile ./tf-webserver_vm/webserver_vm.tf


resource "oci_core_instance" "webserver_instance" {
  # Required
  availability_domain = data.oci_identity_availability_domains.ads.availability_domains[0].name
  compartment_id = var.compartment_id
  shape = "VM.Standard.E2.1.Micro"
  source_details {
    # https://docs.oracle.com/en-us/iaas/images/ 
    # Use Canonical-Ubuntu-22.04-Minimal for your region
    source_id = "<source-ocid>"
    source_type = "image"
  }

  # Optional
  display_name = "ubuntu-webserver-instance"
  create_vnic_details {
    assign_public_ip = true
    # Public subnet
    # Find this in your VCN outputs.  
    subnet_id = "<subnet-ocid>"
  }
  metadata = {
    ssh_authorized_keys = file("<ssh-public-key-path>")
  } 
  preserve_boot_volume = false
}

In [17]:
!code ./tf-webserver_vm/webserver_vm.tf 

In [ ]:
%%writefile ./tf-webserver_vm/outputs.tf


# The "name" of the availability domain to be used for the compute instance.
output "name-of-first-availability-domain" {
  value = data.oci_identity_availability_domains.ads.availability_domains[0].name
}

# Outputs for compute instance
output "public-ip-for-compute-instance" {
  value = oci_core_instance.webserver_instance.public_ip
}


output "private-ip-for-compute-instance" {
  value = oci_core_instance.webserver_instance.private_ip
}

output "instance-name" {
  value = oci_core_instance.webserver_instance.display_name
}

output "instance-OCID" {
  value = oci_core_instance.webserver_instance.id
}

output "instance-region" {
  value = oci_core_instance.webserver_instance.region
}

output "instance-shape" {
  value = oci_core_instance.webserver_instance.shape
}

output "instance-state" {
  value = oci_core_instance.webserver_instance.state
}

output "instance-OCPUs" {
  value = oci_core_instance.webserver_instance.shape_config[0].ocpus
}

output "instance-memory-in-GBs" {
  value = oci_core_instance.webserver_instance.shape_config[0].memory_in_gbs
}

output "time-created" {
  value = oci_core_instance.webserver_instance.time_created
}

In [ ]:
!code ./tf-webserver_vm/outputs.tf 

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-webserver_vm && ls -al) 

In [ ]:
!(cd tf-webserver_vm && tree .) 

In [ ]:
!(cd tf-webserver_vm && tofu init) 

In [ ]:
!(cd tf-webserver_vm && tofu validate)

In [ ]:
!(cd tf-webserver_vm && tofu plan -input=false)

In [ ]:
!(cd tf-webserver_vm && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-webserver_vm && tofu output)

# Configure Servers
First, copy the Database VM SSH private key to the Web Server VM.

On your Ubuntu client, change into the .ssh directory:

```bash
cd
cd .ssh
ls -al
```
You should see the Database VM SSH keys.

Now connect to the Webserver VM using SFTP:
```bash
sftp -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
```
For example:
```bash
sftp -i /home/harley/.ssh/webserver_vm_key ubuntu@168.138.11.254
```
Once connected to the Webserver VM, transfer the private key:
```bash
put database_vm_key
exit
```
Now connect to the Webserver VM using SSH:
```bash
ssh -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
```
For example:
```bash
ssh -i /home/harley/.ssh/webserver_vm_key ubuntu@168.138.11.254
```
Once connected to the Webserver VM, move the private key to .ssh:
```bash
mv database_vm_key .ssh/
```
Next, we'll download files we need to install on the Database VM.

## Get and transfer files for MySQL

Check for newer software before running the following wget commands.

[Index of /debian/pool/main/d/dpkg](http://deb.debian.org/debian/pool/main/d/dpkg/)

[MySQL Community Downloads](https://dev.mysql.com/downloads/mysql/)

Run the following commands in the webserver terminal:

```bash
wget http://deb.debian.org/debian/pool/main/d/dpkg/dpkg-dev_1.22.2_all.deb
wget https://dev.mysql.com/get/Downloads/MySQL-8.3/mysql-server_8.3.0-1ubuntu22.04_amd64.deb-bundle.tar
```
Now connect to the Database VM using SFTP:
```bash
sftp -i <ssh-private-key-path> ubuntu@<your-private-ip-address>
```
For example:
```bash
sftp -i /home/ubuntu/.ssh/database_vm_key ubuntu@10.0.1.134
```
Once connected to the Database VM, transfer the files:
```bash
put dpkg-dev_1.22.2_all.deb 
put mysql-server_8.3.0-1ubuntu22.04_amd64.deb-bundle.tar
exit
```
Now connect to the Database VM using SSH:
```bash
ssh -i <ssh-private-key-path> ubuntu@<your-private-ip-address>
```
For example:
```bash
ssh -i /home/ubuntu/.ssh/database_vm_key ubuntu@10.0.1.134
```

## Set up MySQL
You should now be connected to your Database VM.

First, install some prerequisites:
```bash
sudo su
apt install ./dpkg-dev_1.22.2_all.deb
```
Now install MySQL:
```bash
tar -xvf mysql-server_8.3.0-1ubuntu22.04_amd64.deb-bundle.tar
dpkg-preconfigure mysql-community-server_*.deb
dpkg -i mysql-{common,community-client-plugins,community-client-core,community-client,client,community-server-core,community-server,server}_*.deb
```
Check your installation:
```bash
mysql --version
```
Start MySQL
```bash
systemctl start mysql
```
Configure firewall:
```bash
iptables -I INPUT 6 -m state --state NEW -p tcp --dport 3306 -j ACCEPT
netfilter-persistent save
```
To have MySQL start at boot:
```bash
systemctl enable mysql
```

Open MySQL
```bash
mysql -u root
```
Create a database and a user:
```sql
CREATE DATABASE wordpress;
CREATE USER 'bob'@'%' IDENTIFIED BY 'EO750FAS7F6Tvpzx';
GRANT SELECT, INSERT, UPDATE, DELETE, CREATE, DROP, ALTER ON wordpress.* TO 'bob'@'%';
FLUSH PRIVILEGES;
```
Now use wordpress database:
```sql
USE wordpress;

```

## Set up Apache

Open a new terminal tab, connect to the webserver using SSH, and run the following command in the webserver terminal:
```bash
sudo su

apt update
apt -y install apache2
systemctl restart apache2

iptables -I INPUT 6 -m state --state NEW -p tcp --dport 80 -j ACCEPT
iptables -I INPUT 6 -m state --state NEW -p tcp --dport 443 -j ACCEPT
netfilter-persistent save
```
In the browser, connect to http://your-public-ip-address

## Set up PHP

Run the following command in the webserver terminal:
```bash
apt -y install php libapache2-mod-php
systemctl restart apache2

echo '<?php
  phpinfo();
?>' > /var/www/html/info.php
```
In the browser, connect to http://your-public-ip-address/info.php


## Install phpmyadmin

```bash
apt install phpmyadmin

apt install vim
vim /etc/phpmyadmin/config.inc.php

```

Add the lines below to the bottom of: /etc/phpmyadmin/config.inc.php

```
$i++;
$cfg['Servers'][$i]['host'] = 'HostName:port'; //provide hostname and port if other than default
$cfg['Servers'][$i]['user'] = 'userName';      //user name for your remote server
$cfg['Servers'][$i]['password'] = 'Password';  //password
$cfg['Servers'][$i]['auth_type'] = 'config';   // keep it as config

```

In the browser, connect to http://your-public-ip-address/phpmyadmin


## Setup Wordpress


```bash
apt install wordpress
vim /etc/apache2/sites-available/wordpress.conf
```
Add the following to the file:
```
Alias /blog /usr/share/wordpress
<Directory /usr/share/wordpress>
    Options FollowSymLinks
    AllowOverride Limit Options FileInfo
    DirectoryIndex index.php
    Order allow,deny
    Allow from all
</Directory>
<Directory /usr/share/wordpress/wp-content>
    Options FollowSymLinks
    Order allow,deny
    Allow from all
</Directory>
```
Run the following commands:

```bash
a2ensite wordpress
systemctl reload apache2
```

Configure WordPress to use a MySQL database. 
Open the /etc/wordpress/config-localhost.php file:

```bash
vim /etc/wordpress/config-localhost.php
```

Write the following lines:
```
<?php
define('DB_NAME', 'wordpress');
define('DB_USER', 'bob');
define('DB_PASSWORD', 'EO750FAS7F6Tvpzx');
define('DB_HOST', '<private-ip-address>');
define('WP_CONTENT_DIR', '/usr/share/wordpress/wp-content');
?>
```
```bash
mv /etc/wordpress/config-localhost.php /etc/wordpress/config-<public-ip-address>.php 
```

In the browser, connect to http://your-public-ip-address/blog/wp-admin/install.php


# Reboot
Reboot both VMs and make sure everything still works.
```bash
reboot
```
Check your website here:
```
http://<your-public-ip-address>/index.php
```

# Security

## Webserver
Remove the info.php file:

```bash
rm /var/www/html/info.php
```

## Database

Remove the Database VM SSH private key from the Web Server VM once you have finished configuration.

Don't lose it.

## Two-Factor Authentication (2FA)

Enable 2FA on the webserver.

[Set Up SSH Two-Factor Authentication](https://www.linuxbabe.com/debian/ssh-two-factor-authentication-debian)

When using the instructions, remember these things:

- Run google-authenticator as the ubuntu user, not the root user.
- In Step 2, use the instructions for Public key authentication with 2FA.
- Exit the authenticator app and reopen it to get a fresh code.
- Test your config by logging into the Webserver using SSH in a new tab before you close the tab you are using for configuration.
- DO NOT CLOSE YOUR SSH SESSION UNTIL YOU HAVE VERIFIED FUNCTIONALITY.

YOU REALLY SHOULD ALSO SETUP 2FA FOR WordPress AND phpMyAdmin ALSO.

# Destroy Resources

In [ ]:
!(cd tf-webserver_vm && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-database_vm && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-vcn && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-compartment && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-provider && tofu destroy -input=false -auto-approve)